In [ ]:
from datasets import load_dataset

# Load private dataset
#dataset = load_dataset("mikeriess/LM_Da_test_100_llama-3.3-70b-instruct")
#dataset = load_dataset("mikeriess/LM_Da_test_100_llama-3.1-405b-instruct-fp8")
#dataset = load_dataset("mikeriess/LM_dialogues1")
dataset = load_dataset("mikeriess/LM_dialogues1", revision="0827e2f7f30c6192f71113ab579067c4fbda3b0f")

Generating train split: 100%|██████████| 734/734 [00:00<00:00, 151499.39 examples/s]


In [ ]:
# Convert to pandas and then to dictionary format
df = dataset['train'].to_pandas()
df.head()

,model,conversation_id,orig_messages,synthetic_messages,orig_message_count,synthetic_message_count,message_count_diff,orig_total_length,synthetic_total_length,orig_total_tokens,synthetic_total_tokens
0,meta-llama/Llama-3.1-405B-Instruct-FP8,65536,"[{'order': 1, 'text': 'Go for it: [URL] MP3 fi...","[{'order': 1, 'text': 'Hej, jeg prøver at down...",3,3,0,304,98,57,98
1,meta-llama/Llama-3.1-405B-Instruct-FP8,65538,"[{'order': 1, 'text': '[URL] Little AlterBoy: ...","[{'order': 1, 'text': 'Hej, jeg søger en plugi...",8,8,0,473,151,88,151
2,meta-llama/Llama-3.1-405B-Instruct-FP8,65539,"[{'order': 1, 'text': 'Jeg er begyndt at have ...","[{'order': 1, 'text': 'Hej, jeg håber nogen ka...",1,1,0,432,45,75,45
3,meta-llama/Llama-3.1-405B-Instruct-FP8,65544,"[{'order': 1, 'text': 'Looper og syncer dårlig...","[{'order': 1, 'text': 'Hej, jeg har et problem...",2,2,0,481,70,90,70
4,meta-llama/Llama-3.1-405B-Instruct-FP8,65547,"[{'order': 1, 'text': 'Hej Min kammerat og jeg...","[{'order': 1, 'text': 'Hej, jeg søger hjælp ti...",6,6,0,2796,324,503,324


In [ ]:
print(df.loc[53]['synthetic_messages'])

[{'order': 1, 'text': 'Hej, jeg har købt en elektrisk guitarforstærker i USA, som kun kan køre på 110V. Jeg ved, jeg skal bruge en adapter til at køre den på 230V herhjemme, men jeg er lidt bekymret for, om det vil påvirke lyden eller funktionen. Har du nogen erfaring med lignende udstyr?', 'turn': 1, 'user': 1}
 {'order': 2, 'text': 'Ja, jeg har selv købt en del udstyr i USA, inklusive en guitarforstærker. Jeg har brugt en step-down-transformator til at køre den på 230V, og jeg har ikke oplevet nogen problemer med lyden eller funktionen.', 'turn': 1, 'user': 2}
 {'order': 3, 'text': 'En step-down-transformator? Er det ikke bare en adapter, man skal bruge? Jeg troede, det var nok at bruge en almindelig adapter til at køre udstyret på 230V.', 'turn': 2, 'user': 1}
 {'order': 4, 'text': 'Nej, en adapter er ikke nok i dette tilfælde. En adapter vil kun ændre stikket, men ikke spændingen. En step-down-transformator derimod vil sænke spændingen fra 230V til 110V, så udstyret kan køre sikker

In [ ]:
df.columns

Index(['model', 'conversation_id', 'orig_messages', 'synthetic_messages',
       'orig_message_count', 'synthetic_message_count', 'message_count_diff',
       'orig_total_length', 'synthetic_total_length', 'orig_total_tokens',
       'synthetic_total_tokens'],
      dtype='object')

# Convert to ChatML format

In [ ]:
# Convert dataset to pandas
df = dataset['train'].to_pandas()

# Group by question_id and concatenate conversations
chatml_data = {}
for idx, row in df.iterrows():
    # Create messages list
    messages = []
    for message in row['synthetic_messages']:
        # Determine role based on user field
        # Assuming user 1 is the questioner (user) and user 2 is the answerer (assistant)
        role = "user" if message['user'] == 1 else "assistant"
        
        messages.append({
            "role": role,
            "content": message['text']
        })
    
    # Generate a unique ID if question_id doesn't exist
    question_id = row.get('question_id', f"synthetic_{idx}")
    
    # Store in chatml format
    chatml_data[question_id] = {
        "messages": messages,
        "language": "Danish"  # Assuming all messages are in Danish
    }

# Print sample to verify format
print("Sample conversation:")
sample_id = list(chatml_data.keys())[0]
print(f"\nQuestion ID: {sample_id}")
print(f"Language: {chatml_data[sample_id]['language']}")
print("\nMessages:")
for msg in chatml_data[sample_id]['messages']:
    print(f"{msg['role']}: {msg['content']}\n")

Sample conversation:

Question ID: synthetic_0
Language: Danish

Messages:
user: Hej, jeg søger efter nogle solide højtaler-stande til mine nye højtalere, som vejrer 20 kg hver. Jeg har set på SoundAnchors, men ville gerne høre, om der er andre mærker, der kan anbefales?

assistant: Hej! Ja, SoundAnchors er et godt mærke, men der er også andre muligheder. Jeg har selv haft gode erfaringer med Atacama, som laver nogle rigtig solide og stabile stande. De har også en del forskellige modeller, der passer til forskellige vægtklasser.

user: Tak for tippet! Jeg vil lige kigge på Atacama. Er der noget, jeg skal være særligt opmærksom på, når jeg vælger en stand? Jeg har hørt, at justerbar højde kan være en god ting, men jeg er mere bekymret for stabiliteten.

assistant: Ja, stabilitet er absolut det vigtigste. Du skal sikre dig, at standene kan bære vægten af dine højtalere uden at vakle eller tippe. Justerbar højde kan være praktisk, men det er ikke nødvendigt, hvis du har en fast installati

In [ ]:
chatml_data

{'synthetic_0': {'messages': [{'role': 'user',
    'content': 'Hej, jeg søger efter nogle solide højtaler-stande til mine nye højtalere, som vejrer 20 kg hver. Jeg har set på SoundAnchors, men ville gerne høre, om der er andre mærker, der kan anbefales?'},
   {'role': 'assistant',
    'content': 'Hej! Ja, SoundAnchors er et godt mærke, men der er også andre muligheder. Jeg har selv haft gode erfaringer med Atacama, som laver nogle rigtig solide og stabile stande. De har også en del forskellige modeller, der passer til forskellige vægtklasser.'},
   {'role': 'user',
    'content': 'Tak for tippet! Jeg vil lige kigge på Atacama. Er der noget, jeg skal være særligt opmærksom på, når jeg vælger en stand? Jeg har hørt, at justerbar højde kan være en god ting, men jeg er mere bekymret for stabiliteten.'},
   {'role': 'assistant',
    'content': 'Ja, stabilitet er absolut det vigtigste. Du skal sikre dig, at standene kan bære vægten af dine højtalere uden at vakle eller tippe. Justerbar højde

In [ ]:
print("Sample conversation:")
sample_id = list(chatml_data.keys())[1]
print(f"\nQuestion ID: {sample_id}")
print(f"Language: {chatml_data[sample_id]['language']}")
print("\nMessages:")
for msg in chatml_data[sample_id]['messages']:
    #print(f"{msg['role']}: {msg['content'][:100]}...")
    print(f"{msg['role']}: {msg['content']}"+ "\n")

# Convert to axolotl format

In [ ]:
# Convert to axolotl format
from datasets import Dataset
import json

# Convert conversations to ChatML format
def convert_to_chatml(df):
    formatted_data = []
    
    # Process each row in dataframe
    for _, row in df.iterrows():
        # Extract messages
        messages = []
        
        # Convert synthetic messages to ChatML format
        for msg in row['synthetic_messages']:
            messages.append({
                'role': 'user' if msg['user'] == 1 else 'assistant',
                'content': msg['text']
            })
            
        # Skip empty conversations
        if not messages:
            continue
            
        # Format as ChatML
        formatted_convo = {
            'messages': messages,
            'language': 'Danish'  # All conversations are in Danish
        }
        
        formatted_data.append(formatted_convo)
        
    return formatted_data

# Convert dataframe to ChatML format
chatml_data = convert_to_chatml(df)

# Create HF dataset
dataset = Dataset.from_list(chatml_data)

# Split into train/validation
train_test = dataset.train_test_split(test_size=0.1)

In [ ]:
train_test

DatasetDict({
    train: Dataset({
        features: ['messages', 'language'],
        num_rows: 435
    })
    test: Dataset({
        features: ['messages', 'language'],
        num_rows: 49
    })
})

In [ ]:
# Push to hub
train_test.push_to_hub("mikeriess/LM_dialogues1")

print(f"Uploaded {len(chatml_data)} conversations to HF hub")

# Verify format of first example
print("\nSample conversation format:")
print(json.dumps(chatml_data[0], indent=2))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploaded 484 conversations to HF hub

Sample conversation format:
{
  "messages": [
    {
      "role": "user",
      "content": "Hej, jeg s\u00f8ger efter nogle solide h\u00f8jtaler-stande til mine nye h\u00f8jtalere, som vejrer 20 kg hver. Jeg har set p\u00e5 SoundAnchors, men ville gerne h\u00f8re, om der er andre m\u00e6rker, der kan anbefales?"
    },
    {
      "role": "assistant",
      "content": "Hej! Ja, SoundAnchors er et godt m\u00e6rke, men der er ogs\u00e5 andre muligheder. Jeg har selv haft gode erfaringer med Atacama, som laver nogle rigtig solide og stabile stande. De har ogs\u00e5 en del forskellige modeller, der passer til forskellige v\u00e6gtklasser."
    },
    {
      "role": "user",
      "content": "Tak for tippet! Jeg vil lige kigge p\u00e5 Atacama. Er der noget, jeg skal v\u00e6re s\u00e6rligt opm\u00e6rksom p\u00e5, n\u00e5r jeg v\u00e6lger en stand? Jeg har h\u00f8rt, at justerbar h\u00f8jde kan v\u00e6re en god ting, men jeg er mere bekymret for stabilite